# Salary.com
extract salary information for a data scientist job in the 317 largest us cities

In [73]:
import csv
import json
import requests
from bs4 import BeautifulSoup

In [87]:
template = 'https://www.salary.com/research/salary/listing/data-scientist-salary/{}'

In [85]:
# get list of largest us cities
with open('largest_cities.csv', 'r', newline='', encoding='utf-8') as f:
    reader = csv.reader(f)
    city_list = []
    for city in reader:
        city_list.extend(city)
        
city_list = [city.replace(' ', '-') for city in city_list]

In [88]:
def extract_page_data(html_text):
    """Extract all job data from raw html"""
    soup = BeautifulSoup(response.text, 'lxml')
    script = soup.find_all('script', {'type': 'application/ld+json'})[1]
    json_data = json.loads(script.contents[0])
    
    # job details
    title = json_data.get('name')
    location = json_data.get('occupationLocation')[0]['name']
    responsibilities = json_data.get('responsibilities')
    
    # salary
    base_salary = json_data.get('estimatedSalary')[0] # base
    ntile10 = base_salary.get('percentile10')
    ntile25 = base_salary.get('percentile25')
    ntile50 = base_salary.get('median')
    ntile75 = base_salary.get('percentile75')
    ntile90 = base_salary.get('percentile90')
    
    record = {
        'title': title,
        'location': location,
        'responsibilities': responsibilities,
        'ntile10': ntile10,
        'ntile25': ntile25,
        'ntile50': ntile50,
        'ntile75': ntile75,
        'ntile90': ntile90
    }
    return record

In [97]:
records = []

for city in city_list[316:]:
    url = template.format(city)
    response = requests.get(url)
    if response.reason == 'OK':
        record = extract_page_data(response.text)
        if record:
            records.append(record)
    else:
        continue

In [105]:
fieldnames = list(records[0].keys())

with open('data-scientist-salary.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(records)

In [106]:
records[0]

{'title': 'Data Scientist',
 'location': 'New York, NY',
 'responsibilities': 'Create data tools for analytics and data scientist team members that assist them in building and optimizing our product into an innovative industry leader.',
 'ntile10': '123501',
 'ntile25': '138561',
 'ntile50': '155102',
 'ntile75': '171349',
 'ntile90': '186141'}